# Group C Jupyter Notebook
##   A PYNQ Theremin
 
This is the Jupyter Notebook for using a PYNQ-Z2 board as a Theremin. A theremin is a musical instrument is that has its sound changed from two inputs, pitch and volume. In this Notebook, a Grove Ultrasonic Ranger is used to measure the distance from the sensor and change the pitch accordingly. The volume is controlled from the gain sliders. More in depth explanation can be found in references.


<img src="diagram_theremin.png" height = "600" width = "600">
Figure 1. Hardware and Software Flowchart





### Numerically Controlled Oscillator (NCO)

A NCO (Figure 2) is a signal generator which digitally creates a synchronous, waveform which is discrete-valued and discrete-timed. The output is usually a sinusoidal waveform and they are often used with a digital-to-analog converter to create a digital audio synthesizer.

<img src="nco1.png" height = "600" width = "600">
Figure 2. Basic NCO diagram [1]


It takes an input value (step_size) which determines how quickly the address is accumulated,and hence the frequency of the generated sine wave. This value goes through an accumulator (Figure 3) and is a fixed point number. It has n whole bits and b fractional bits. Increasing the fractional bit provides greater precision in the frequency.




<img src="nco2.png" height = "500" width = "500">
Figure 3. Accumulator diagram [1]

### Grove Ultrasonic Ranger (USR)

The USR (Figure 4) is a device that connects to the G1 port of the PYNQ Grove Adapter which connects to the PMODA port of the PYNQ-Z2 board. It is a "non-contact distance measurement module" [2] that emits and detects ultrasonic sound waves which in this case are used for measuring distances. The distance measured changes the pitch of the outputed sound wave.

<img src="usr1.jpg" height = "600" width = "600">
Figure 4. USR comparison diagram. Model used on the left [2]

The model used in Figure 4 has some key features and conections. When compared to other ultrasonic sensor modules (like the one on the right in the Figure 4) it has a single-chip microcomputer, and the ECHO signal along with the TRIG signal share one pin.

In Figure 5:

* <font color='green'>Green</font>  - Audio Codec
* <font color='red'>Red</font> - NCO
* <font color='orange'>Yellow</font> - 10 MHz Clock Wizard
* <font color='purple'>Purple</font> - ZYNQ7 Processing System


<img src="full_block_diagram.png" height = "600" width = "600">
Figure 5. Full Vivado Block Design




The NCO used the AXI interface to receive the Left/Right Channel Gains and the Step Size. The three outputs of the NCO block (Figure 6) and the "codec_address[1:0]" of the Audio Codec block are fed into the corresponding ports of the PYNQ-Z2 board in order to have audible tone as an output.





<img src="nco_block_diagram.png" height = "600" width = "600">
Figure 6. NCO and Audio Codec in Block Design





### Libraries

First the correct libraries are imported and the correct bitstream file is used for the overlay.

In [1]:
import pynq
from pynq import Overlay
from pynq import Clocks
import ipywidgets as widgets
import _thread
from IPython.display import clear_output

theremin = Overlay("theremin_group_c.bit") # hwh is parsed here

Running this cell will display the hierarchies and the IPs in the system.

In [2]:
?theremin

### USR Code

Cell below was resourced [3] where it was provided with the PYNQ board as an example. Written in C.  

The ultrasonic sensor must be attached to the G1 connector of the Pmod Grove adapter, and the adapter is connected to PMODA.

Some values for the timer controller registers are manipulated and their definitions are shown below:

|Register name |Register functionality               |Register value |
|:-------------|:------------------------------------|:----------------|
|TCSR0         |Timer 0 Control and Status Register  |0x00             |
|TLR0          |Timer 0 Load Register                |0x04             |
|TCR0          |Timer 0 Counter Register             |0x08             |
|TCSR1         |Timer 1 Control and Status Register  |0x10             |
|TLR1          |Timer 1 Load Register                |0x14             |
|TCR1          |Timer 1 Counter Register             |0x18             |

In [3]:
%%microblaze theremin.iop_pmoda

#include "xparameters.h"
#include "xtmrctr.h"
#include "gpio.h"
#include "timer.h"
#include <pmod_grove.h>

#define TCSR0 0x00
#define TLR0 0x04
#define TCR0 0x08
#define TCSR1 0x10
#define TLR1 0x14
#define TCR1 0x18
#define MAX_COUNT 0xFFFFFFFF

void create_10us_pulse(gpio usranger){ 
    gpio_set_direction(usranger, GPIO_OUT); //sets the USR to output
    gpio_write(usranger, 0); //outputs no sound waves for 2 us
    delay_us(2);  
    gpio_write(usranger, 1); //outputs a sound wave for 10 us
    delay_us(10);
    gpio_write(usranger, 0); //sound wave output is 0
}

void configure_as_input(gpio usranger){
    gpio_set_direction(usranger, GPIO_IN); //sets the USR as an input to detect the reflection
}

unsigned int capture_duration(gpio usranger){
    unsigned int count1, count2;
    count1=0;
    count2=0;
    XTmrCtr_WriteReg(XPAR_TMRCTR_0_BASEADDR, 0, TLR0, 0x0);
    XTmrCtr_WriteReg(XPAR_TMRCTR_0_BASEADDR, 0, TCSR0, 0x190);
    while(!gpio_read(usranger));
    count1=XTmrCtr_ReadReg(XPAR_TMRCTR_0_BASEADDR, 0, TCR0);
    while(gpio_read(usranger));
    count2=XTmrCtr_ReadReg(XPAR_TMRCTR_0_BASEADDR, 0, TCR0);
    if(count2 > count1) {
        return (count2 - count1);
    } else {
        return((MAX_COUNT - count1) + count2);  
    }
}

unsigned int read_raw(){ //read function that reads the duration
    gpio usranger;
    usranger = gpio_open(PMOD_G1_A);
    create_10us_pulse(usranger);
    configure_as_input(usranger);
    return capture_duration(usranger);
}

### IP variable assignment

Take the audio codec and NCO IPs and assign them to variables for ease of use. Also configure the audio codec. 

In [4]:
pAudio = theremin.audio_codec_ctrl_0
pAudio.configure()
nco = theremin.nco_ip_0

### NCO Properties

Set the system and NCO clock frequency, both taken from System Generator. The sampling frequency is calculated from these values and the LUT depth is also assigned. [4] [5]

In [5]:
sys_clk_freq = 100000000 #100MHz
nco_clk_freq = 1024 #32*32 from the delays in system generator
fs = (sys_clk_freq / nco_clk_freq) #sampling frequency
N = 256 #2^8 LUT depth

### Functions
* **read_distance_cm** - Calculations were resourced from [1] as previously stated. The number 58 is from the follwing fomrula and time is the variable measured:

$d = \frac{1}{2}*340m/s * 10^{-6} * time$

$d$ is in meters, so $100 * d$ for centimeters.

* **step_size_calculation** - Definition of the step_size_calculation() function takes place, which will be used to assign a correct value to the NCO. [2]


* **setGains** - Create the setGains function to set the gain values received from the sliders. [2]


In [6]:
def read_distance_cm():
    raw_value = read_raw()
    clk_period_ns = int(1000 / Clocks.fclk0_mhz)
    num_microseconds = raw_value * clk_period_ns * 0.001
    distance = num_microseconds/58
    if num_microseconds * 0.001 > 30:
        return 500
    else:
        return distance

def step_size_calculation(fd):

    return int(((N * fd) / fs)*(2**20)) #20 fractional bit in the input

def setGains(Lch, Rch):
    try:
        nco.write(0x0, int(Lch*(2**3)))  # left Channel 3 fractional bits
        nco.write(0x4, int(Rch*(2**3)))  # Right Channel 3 fractional bits
    except Exception:
        import traceback
        print(traceback.format_exc())

### Widgets

Import the widgets library and configure both sliders. [4]

In [7]:
LR_Slider = widgets.FloatSlider(
    value=1,
    min=0,
    max=1,
    step=0.125,
    description='L/R Gain',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
)

Duration_Slider = widgets.FloatSlider(
    value=1,
    min=5,
    max=30,
    step=1,
    description='Duration (s)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
)

### Output

Run the following two cells and adjust the sliders to change the duration of the theremin sound and volume <font color='red'>before</font> running the final cell.

In [12]:
Duration_Slider

FloatSlider(value=30.0, continuous_update=False, description='Duration (s)', max=30.0, min=5.0, readout_format…

In [13]:
LR_Slider

FloatSlider(value=0.5, continuous_update=False, description='L/R Gain', max=1.0, readout_format='.3f', step=0.…

Used a for-loop to write continuously to the NCO and have a continuous output. Started two threads for the audio output, one for the audio codec and one for the gain sliders. This in turn gives an audio output which can be heard. It also outputs the measurements taken. Once the loop finishes, the output of the cell is cleared and ready to be ran again.

In [15]:
setGains(LR_Slider.value,LR_Slider.value)
for i in range(int(Duration_Slider.value * 20)):
    nco.write(0x8, step_size_calculation(read_distance_cm()*25)) #Step Size input desired frequency
    print('distance: {:.2f} cm'.format((read_distance_cm())))

    try:
        _thread.start_new_thread(pAudio.bypass, (0.1,))
    except Exception:
        import traceback
        print(traceback.format_exc())


for i in range(10):
    clear_output(wait=True)
    print("Finished. Great song!")

Finished. Great song!


# References

[1] Further VHDL and FPGA Design - Lecture Week 5 Chapter 5: Numerically Controlled Oscillators 

[2] Grove - Ultrasonic Distance Sensor https://www.seeedstudio.com/Grove-Ultrasonic-Distance-Sensor.html

[3] Provided on PYNQ board http://192.168.2.99:9090/notebooks/base/pmod/pmod_grove_usranger.ipynb

[4] Dr. Louise Crocket Audio Example https://classes.myplace.strath.ac.uk/mod/resource/view.php?id=1696059

[5] Audio Codec Datasheet https://www.analog.com/media/en/technical-documentation/data-sheets/ADAU1761.pdf